## Single Household demonstration
This notebook demonstrates basic usage of the powergrid model implemented in 'src/grid.py'. We will consider a single household that consumes energy, such as a realistic user. For the modeling of energy usage and production we use two datasets from UK Networks (more detailed description elsewhere).

In [1]:
# Import all needed libraries
import pandas as pd
from src.grid import Grid
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### Import datasets for consumption and production

In [2]:
# Test import of consumption data
df_cons = pd.read_csv('data/consumption_ausgrid.csv')
df_cons['Unnamed: 0'] = pd.to_datetime(df_cons['Unnamed: 0'])
df_cons = df_cons.set_index('Unnamed: 0')

# Test import of production data
df_prod = pd.read_csv('data/generation_ausgrid.csv')
df_prod['Unnamed: 0'] = pd.to_datetime(df_prod['Unnamed: 0'])
df_prod = df_prod.set_index('Unnamed: 0')

### Single household - no PV - no storage
For simplicity we will first consider one house with no storage and no pv installed. Thus this is the basic example of a single household connected to the powergrid and should just return the basic electricity usage, i.e. 
consumption in kWh. We choose the dataset of the AUSGRID company, which provides a full year of data.

In [3]:
# Reduce the dataset to example described above
df_cons_red = df_cons.iloc[:,100] # Take only consumption of house 100
df_prod_red = pd.DataFrame() # Empty 
# Convert to numpy arrays for Grid.simulate()
data_cons_red = df_cons_red.values
data_prod_red = df_prod_red.values

Now we can import the model from the src/grid.py and set it up with the properties needed for the simulation describe above. Due to the implementation, we have to set the number of storages to one, but simply set its capacity to zero. Hence it is the same situation as no storage system at all.

In [4]:
g_nopv = Grid(num_houses=1,num_storages=1,max_capacity=0,num_pvtypes=0)
g_nopv.set_costs() # Have to that before running
res_nopv = g_nopv.simulate(data_cons=data_cons_red,data_prod=data_prod_red)
print(res_nopv)

{'import_grid_kwh': 3834.486000000009, 'export_grid_kwh': 0.0, 'pv_production_kwh': 0.0, 'setup_cost_storage': 0, 'setup_cost_pv': 0.0, 'cost_import_grid': 958.6215000000022, 'reward_export_grid': 0.0}


We can observe that this particular household imported $3834$ kWh from the energy grid within 1 year. Since it has no energy production by PV, the export in the energy grid is zero. 

### Single household - with PV - no storage

So the next thing is to **attach a PV panel** to the house and check how the imported energy get reduced.

In [5]:
df_cons_red = df_cons.iloc[:,100] # choose house 100
df_prod_red = df_prod['100'] # choose pv panel of house 100
# Conversion to numpy arrays
data_cons_red = df_cons_red.values
data_prod_red = df_prod_red.values

# Setting up a model for this task and simulate it
g_pv = Grid(num_houses=1,num_storages=1,max_capacity=0,num_pvtypes=1)
g_pv.set_costs()
g_pv.mutate(num_house=0,storage_connection=False,pv_type=True) # Currently this turns on the pv on house 0
res_pv = g_pv.simulate(data_cons=data_cons_red,data_prod=data_prod_red)
print(res_pv)

{'import_grid_kwh': 2884.7030000000086, 'export_grid_kwh': 865.9189999999977, 'pv_production_kwh': 1815.7020000000166, 'setup_cost_storage': 0, 'setup_cost_pv': 1400.0, 'cost_import_grid': 721.1757500000022, 'reward_export_grid': 86.59189999999978}


We can see that there is some energy production by the pv panels and there is some export to the energy grid at times when the house does not need energy. Another thing we observe is that the the energy used by the house is exactly the same. Hence our implementation seems to be reasonable.

In [6]:
round(res_pv['import_grid_kwh'] + res_pv['pv_production_kwh'] - res_pv['export_grid_kwh']) == round(res_nopv['import_grid_kwh'])

True

### Single household - PV - storage
The last thing to implement is an energy storage. This storage takes energy from the PV panels and stores it until the house needs energy. Then it can take the energy of the storage instead of using it from the grid. We use here a storage system of $13.5$ kWh, which is the basic Powerwall model from Tesla.

In [9]:
g_pv_storage = Grid(num_houses=1,num_storages=1,max_capacity=13.5,num_pvtypes=1)
g_pv_storage.set_costs()
g_pv_storage.mutate(num_house=0,storage_connection=False,pv_type=True)
res_pv_storage = g_pv_storage.simulate(data_cons=data_cons_red,data_prod=data_prod_red)
print(res_pv_storage)

{'import_grid_kwh': 2039.033999999996, 'export_grid_kwh': 20.250000000000014, 'pv_production_kwh': 1815.7020000000166, 'setup_cost_storage': 6750.0, 'setup_cost_pv': 1400.0, 'cost_import_grid': 509.758499999999, 'reward_export_grid': 2.0250000000000017}


As expected the included storage system reduces the dependency on the grid. The reduction of the import will reduce the amount of money we have to pay, to the provider. Another thing is that storing energy and using it later is assumed to be more efficient than selling and buying new energy afterwards. However all of these statements are hypotheses and we can later check them by giving a cost to every hardware piece and each kWh.